#GRUPO 04 - Integrantes

- Agustin Braida
- Ignacio Carrera
- Matias Etchegoyen
- Agustin Trombetta

#Exploración inicial y preprocesamiento

In [1]:
!pip install gensim --quiet
!python -m spacy download es_core_news_sm
!pip install keras-tuner --quiet
!pip install langdetect
!pip install joblib

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 2.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.6/60.6 kB 1.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 26.7/26.7 MB 27.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.3/18.3 MB 47.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 38.6/38.6 MB 15.3 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
thinc 8.3.6 requires numpy<3.0.0,>=2.0.0, but you have numpy 1.26.4 which is incompatible.
tsfresh 0.21.0 requires scipy>=1.14.0; python_version >= "3.10", but you have scipy 1.13.1 which is incompatible.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.9/12.9 MB 99.8 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('es_core_news_sm')
⚠ Restart to reload dependencies
If you 

Imports

In [1]:
from google.colab import drive
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import re
from sklearn.feature_extraction.text import TfidfVectorizer
import spacy
from gensim.models import Word2Vec
import xgboost as xgb
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split, GridSearchCV, RandomizedSearchCV
from sklearn.metrics import classification_report, f1_score
from sklearn.naive_bayes import MultinomialNB
import keras_tuner as kt
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, BatchNormalization
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
from langdetect.lang_detect_exception import LangDetectException
from joblib import dump, load

Carga de datos

In [2]:
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
path = '/content/drive/MyDrive/data_movies/'

train_df = pd.read_csv(path + 'train.csv')
test_df = pd.read_csv(path + 'test.csv')

Tamaños

In [4]:
print("Train shape:", train_df.shape)
print("Test shape:", test_df.shape)

Train shape: (50000, 3)
Test shape: (8599, 2)


Primeras filas

In [5]:
train_df.head()

,ID,review_es,sentimiento
0,0,Uno de los otros críticos ha mencionado que de...,positivo
1,1,Una pequeña pequeña producción.La técnica de f...,positivo
2,2,Pensé que esta era una manera maravillosa de p...,positivo
3,3,"Básicamente, hay una familia donde un niño peq...",negativo
4,4,"El ""amor en el tiempo"" de Petter Mattei es una...",positivo


Mapeo del sentimiento

In [6]:
train_df['sentimiento_binario'] = train_df['sentimiento'].map({'negativo': 0, 'positivo': 1})

Hay balance?

In [7]:
train_df['sentimiento'].value_counts(normalize=True)

,proportion
sentimiento,
positivo,0.5
negativo,0.5


Hay nulos?

In [8]:
train_df.isnull().sum()

,0
ID,0
review_es,0
sentimiento,0
sentimiento_binario,0


Descripción de las longitudes en caracteres

In [9]:
train_df['review_len'] = train_df['review_es'].str.len()
train_df.groupby("sentimiento")['review_len'].describe()

,count,mean,std,min,25%,50%,75%,max
sentimiento,,,,,,,,
negativo,25000.0,1382.77380,994.097890,39.0,755.0,1041.0,1685.00,8587.0
positivo,25000.0,1413.34268,1079.324047,64.0,738.0,1032.0,1733.25,13465.0


In [10]:
train_df["word_count"] = train_df["review_es"].apply(lambda x: len(x.split()))
train_df.groupby("sentimiento")["word_count"].describe()

,count,mean,std,min,25%,50%,75%,max
sentimiento,,,,,,,,
negativo,25000.0,234.93224,172.630613,6.0,125.0,173.0,289.0,1474.0
positivo,25000.0,239.53512,186.147551,10.0,122.0,172.0,296.0,2450.0


Limpieza de texto

In [11]:
def limpiar_texto(texto):
    texto = texto.lower()
    texto = re.sub(r"http\S+", "", texto)              # Eliminar URLs
    texto = re.sub(r"[^\w\sáéíóúñü]", "", texto)        # Eliminar signos de puntuación
    texto = re.sub(r"\s+", " ", texto)                 # Eliminar espacios múltiples
    return texto.strip()

train_df['review_clean'] = train_df['review_es'].apply(limpiar_texto)
test_df['review_clean'] = test_df['review_es'].apply(limpiar_texto)

In [12]:
train_df.head()

,ID,review_es,sentimiento,sentimiento_binario,review_len,word_count,review_clean
0,0,Uno de los otros críticos ha mencionado que de...,positivo,1,2004,345,uno de los otros críticos ha mencionado que de...
1,1,Una pequeña pequeña producción.La técnica de f...,positivo,1,1061,161,una pequeña pequeña producciónla técnica de fi...
2,2,Pensé que esta era una manera maravillosa de p...,positivo,1,989,173,pensé que esta era una manera maravillosa de p...
3,3,"Básicamente, hay una familia donde un niño peq...",negativo,0,790,137,básicamente hay una familia donde un niño pequ...
4,4,"El ""amor en el tiempo"" de Petter Mattei es una...",positivo,1,1406,237,el amor en el tiempo de petter mattei es una p...


Bag of Words

In [13]:
vectorizer = TfidfVectorizer(max_features=10000, ngram_range=(1,2))  # se puede ajustar

X_train = vectorizer.fit_transform(train_df['review_clean'])
Y_train = train_df['sentimiento_binario']

X_test = vectorizer.transform(test_df['review_clean'])

In [14]:
print("Shape X_train:", X_train.shape)

Shape X_train: (50000, 10000)


Análisis reviews en otro idioma

In [15]:
from langdetect import detect

In [16]:
def detectar_idioma(texto):
    try:
        return detect(texto)
    except LangDetectException:
        return "error"

In [17]:
train_df['idioma'] = train_df['review_es'].apply(detectar_idioma)
train_limpio = train_df[train_df['idioma'] == 'es'].drop(columns='idioma')

test_df['idioma'] = test_df['review_es'].apply(detectar_idioma)
test_limpio = test_df[test_df['idioma'] == 'es'].drop(columns='idioma')

In [18]:
print("Train shape:", train_df.shape)
print("Test shape:", test_df.shape)

Train shape: (50000, 8)
Test shape: (8599, 4)


In [19]:
print("Train shape:", train_limpio.shape)
print("Test shape:", test_limpio.shape)

Train shape: (48181, 7)
Test shape: (8598, 3)


In [20]:
train_limpio.to_csv('train_limpio.csv', index=False)

Embeddings (Word2Vec)

In [135]:
corpus = [review.split() for review in train_df['review_clean']]

w2v_model = Word2Vec(sentences=corpus, vector_size=100, window=5, min_count=2, workers=4, sg=1, seed=42)

def texto_a_vector(texto, model, vector_size=100):
    palabras = texto.split()
    vectores = [model.wv[word] for word in palabras if word in model.wv]
    if len(vectores) == 0:
        return np.zeros(vector_size)
    return np.mean(vectores, axis=0)

X_train_em = np.array([texto_a_vector(texto, w2v_model) for texto in train_df['review_clean']])
Y_train_em = train_df['sentimiento_binario']

X_test_em = np.array([texto_a_vector(texto, w2v_model) for texto in test_df['review_clean']])

In [136]:
# embeddings (word2vec) con dataset con reviews filradas
corpus = [review.split() for review in train_limpio['review_clean']]

w2v_model = Word2Vec(sentences=corpus, vector_size=100, window=5, min_count=2, workers=4, sg=1, seed=42)

def texto_a_vector(texto, model, vector_size=100):
    palabras = texto.split()
    vectores = [model.wv[word] for word in palabras if word in model.wv]
    if len(vectores) == 0:
        return np.zeros(vector_size)
    return np.mean(vectores, axis=0)

X_train_em2 = np.array([texto_a_vector(texto, w2v_model) for texto in train_limpio['review_clean']])
Y_train_em2 = train_limpio['sentimiento_binario']

X_test_em2 = np.array([texto_a_vector(texto, w2v_model) for texto in test_df['review_clean']])

# XGBoost

Dividimos el train, en train y validation

In [ ]:
X_tr, X_val, y_tr, y_val = train_test_split(
    X_train, Y_train, test_size=0.2, random_state=42, stratify=Y_train
)

Definimos el modelo

In [ ]:
xgb_clf = xgb.XGBClassifier(use_label_encoder=False, eval_metric='logloss', random_state=42)

Busqueda de hiperparametros

In [ ]:
param_distributions = {
    'n_estimators': [300, 500],
    'max_depth': [4, 6],
    'learning_rate': [0.05, 0.1],
    'subsample': [0.8, 1.0],
    'colsample_bytree': [0.8, 1.0],
}

In [ ]:
random_search = RandomizedSearchCV(
    estimator=xgb_clf,
    param_distributions=param_distributions,
    n_iter=2,
    scoring='f1',
    cv=3,
    verbose=2,
    random_state=42,
    n_jobs=1
)

In [ ]:
random_search.fit(X_tr, y_tr)

Fitting 3 folds for each of 2 candidates, totalling 6 fits


/usr/local/lib/python3.11/dist-packages/xgboost/core.py:158: UserWarning: [16:55:07] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)


[CV] END colsample_bytree=1.0, learning_rate=0.1, max_depth=6, n_estimators=300, subsample=1.0; total time= 8.6min


/usr/local/lib/python3.11/dist-packages/xgboost/core.py:158: UserWarning: [17:03:46] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)


[CV] END colsample_bytree=1.0, learning_rate=0.1, max_depth=6, n_estimators=300, subsample=1.0; total time= 8.8min


...

[CV] END colsample_bytree=0.8, learning_rate=0.1, max_depth=6, n_estimators=500, subsample=1.0; total time=11.4min


/usr/local/lib/python3.11/dist-packages/xgboost/core.py:158: UserWarning: [17:44:02] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)


[CV] END colsample_bytree=0.8, learning_rate=0.1, max_depth=6, n_estimators=500, subsample=1.0; total time=11.4min


/usr/local/lib/python3.11/dist-packages/xgboost/core.py:158: UserWarning: [17:55:26] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)


RandomizedSearchCV(cv=3,
                   estimator=XGBClassifier(base_score=None, booster=None,
                                           callbacks=None,
                                           colsample_bylevel=None,
                                           colsample_bynode=None,
                                           colsample_bytree=None, device=None,
                                           early_stopping_rounds=None,
                                           enable_categorical=False,
                                           eval_metric='logloss',
                                           feature_types=None, gamma=None,
                                           grow_policy=None,
                                           importance_type=None,
                                           interaction_constraints=None,
                                           learning...
                                           min_child_weight=None, missing=nan,
                                           monotone_constraints=None,
                                           multi_strategy=None,
                                           n_estimators=None, n_jobs=None,
                                           num_parallel_tree=None,
                                           random_state=42, ...),
                   n_iter=2, n_jobs=1,
                   param_distributions={'colsample_bytree': [0.8, 1.0],
                                        'learning_rate': [0.05, 0.1],
                                        'max_depth': [4, 6],
                                        'n_estimators': [300, 500],
                                        'subsample': [0.8, 1.0]},
                   random_state=42, scoring='f1', verbose=2)

In [ ]:
print("Mejores hiperparámetros:", random_search.best_params_)
print("Mejor F1:", random_search.best_score_)

Mejores hiperparámetros: {'subsample': 1.0, 'n_estimators': 500, 'max_depth': 6, 'learning_rate': 0.1, 'colsample_bytree': 0.8}
Mejor F1: 0.8524697854000154


Evaluamos

In [ ]:
best_model = random_search.best_estimator_

y_tr_pred = best_model.predict(X_tr)
print("Train classification report:\n", classification_report(y_tr, y_tr_pred))
print("Train F1 Score:", f1_score(y_tr, y_tr_pred))

y_val_pred = best_model.predict(X_val)
print("\nValidation classification report:\n", classification_report(y_val, y_val_pred))
print("Validation F1 Score:", f1_score(y_val, y_val_pred))

Train classification report:
               precision    recall  f1-score   support

           0       0.97      0.95      0.96     20000
           1       0.95      0.97      0.96     20000

    accuracy                           0.96     40000
   macro avg       0.96      0.96      0.96     40000
weighted avg       0.96      0.96      0.96     40000

Train F1 Score: 0.9602564102564103

Validation classification report:
               precision    recall  f1-score   support

           0       0.87      0.84      0.85      5000
           1       0.85      0.87      0.86      5000

    accuracy                           0.86     10000
   macro avg       0.86      0.86      0.86     10000
weighted avg       0.86      0.86      0.86     10000

Validation F1 Score: 0.8588860460539264


Predecimos

In [ ]:
y_test_pred = best_model.predict(X_test)
y_test_pred_labels = np.where(y_test_pred == 1, 'positivo', 'negativo')

In [ ]:
submission = pd.DataFrame({
    'ID': test_df['ID'],
    'sentimiento': y_test_pred_labels
})

submission.to_csv('xgboost_submission.csv', index=False)

# XGBoost v2.0

Dividimos el train, en train y validation

In [ ]:
X_tr, X_val, y_tr, y_val = train_test_split(
    X_train, Y_train, test_size=0.2, random_state=42, stratify=Y_train
)

Definimos el modelo

In [ ]:
xgb_clf2 = xgb.XGBClassifier(use_label_encoder=False, eval_metric='logloss', random_state=42)

Busqueda de hiperparametros

In [ ]:
param_distributions2 = {
    'n_estimators': [300, 400],
    'max_depth': [3, 4, 5],                # árboles menos profundos
    'learning_rate': [0.05, 0.1],
    'subsample': [0.8, 1.0],
    'colsample_bytree': [0.8, 1.0],
    'gamma': [0, 0.5, 1],                  # mínimo de pérdida para dividir nodos
    'reg_alpha': [0, 0.1, 0.5],            # L1 regularization
    'reg_lambda': [1, 1.5, 2],             # L2 regularization
}

In [ ]:
random_search2 = RandomizedSearchCV(
    estimator=xgb_clf2,
    param_distributions=param_distributions2,
    n_iter=2,          # probamos solo 5 combinaciones para ser rápidos
    scoring='f1',
    cv=3,
    verbose=2,
    random_state=42,
    n_jobs=1
)

In [ ]:
random_search2.fit(X_tr, y_tr)

Fitting 3 folds for each of 2 candidates, totalling 6 fits


/usr/local/lib/python3.11/dist-packages/xgboost/core.py:158: UserWarning: [18:19:56] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)


[CV] END colsample_bytree=1.0, gamma=1, learning_rate=0.05, max_depth=4, n_estimators=300, reg_alpha=0.1, reg_lambda=2, subsample=0.8; total time= 5.1min


/usr/local/lib/python3.11/dist-packages/xgboost/core.py:158: UserWarning: [18:25:00] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)


[CV] END colsample_bytree=1.0, gamma=1, learning_rate=0.05, max_depth=4, n_estimators=300, reg_alpha=0.1, reg_lambda=2, subsample=0.8; total time= 5.1min


...

[CV] END colsample_bytree=1.0, gamma=0, learning_rate=0.1, max_depth=5, n_estimators=400, reg_alpha=0.5, reg_lambda=1.5, subsample=0.8; total time= 8.9min


/usr/local/lib/python3.11/dist-packages/xgboost/core.py:158: UserWarning: [18:52:31] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)


[CV] END colsample_bytree=1.0, gamma=0, learning_rate=0.1, max_depth=5, n_estimators=400, reg_alpha=0.5, reg_lambda=1.5, subsample=0.8; total time= 8.5min


/usr/local/lib/python3.11/dist-packages/xgboost/core.py:158: UserWarning: [19:01:03] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)


RandomizedSearchCV(cv=3,
                   estimator=XGBClassifier(base_score=None, booster=None,
                                           callbacks=None,
                                           colsample_bylevel=None,
                                           colsample_bynode=None,
                                           colsample_bytree=None, device=None,
                                           early_stopping_rounds=None,
                                           enable_categorical=False,
                                           eval_metric='logloss',
                                           feature_types=None, gamma=None,
                                           grow_policy=None,
                                           importance_type=None,
                                           interaction_constraints=None,
                                           learning...
                                           multi_strategy=None,
                                           n_estimators=None, n_jobs=None,
                                           num_parallel_tree=None,
                                           random_state=42, ...),
                   n_iter=2, n_jobs=1,
                   param_distributions={'colsample_bytree': [0.8, 1.0],
                                        'gamma': [0, 0.5, 1],
                                        'learning_rate': [0.05, 0.1],
                                        'max_depth': [3, 4, 5],
                                        'n_estimators': [300, 400],
                                        'reg_alpha': [0, 0.1, 0.5],
                                        'reg_lambda': [1, 1.5, 2],
                                        'subsample': [0.8, 1.0]},
                   random_state=42, scoring='f1', verbose=2)

In [ ]:
print("Mejores hiperparámetros:", random_search2.best_params_)
print("Mejor F1:", random_search2.best_score_)

Mejores hiperparámetros: {'subsample': 0.8, 'reg_lambda': 1.5, 'reg_alpha': 0.5, 'n_estimators': 400, 'max_depth': 5, 'learning_rate': 0.1, 'gamma': 0, 'colsample_bytree': 1.0}
Mejor F1: 0.8476045049939289


Evaluamos

In [ ]:
best_model2 = random_search2.best_estimator_

y_tr_pred = best_model2.predict(X_tr)
print("Train classification report:\n", classification_report(y_tr, y_tr_pred))
print("Train F1 Score:", f1_score(y_tr, y_tr_pred))

y_val_pred = best_model2.predict(X_val)
print("\nValidation classification report:\n", classification_report(y_val, y_val_pred))
print("Validation F1 Score:", f1_score(y_val, y_val_pred))

Train classification report:
               precision    recall  f1-score   support

           0       0.94      0.90      0.92     20000
           1       0.91      0.94      0.93     20000

    accuracy                           0.92     40000
   macro avg       0.92      0.92      0.92     40000
weighted avg       0.92      0.92      0.92     40000

Train F1 Score: 0.9252157708905453

Validation classification report:
               precision    recall  f1-score   support

           0       0.87      0.83      0.85      5000
           1       0.84      0.87      0.85      5000

    accuracy                           0.85     10000
   macro avg       0.85      0.85      0.85     10000
weighted avg       0.85      0.85      0.85     10000

Validation F1 Score: 0.854405006355725


Predecimos

In [ ]:
y_test_pred2 = best_model2.predict(X_test)
y_test_pred_labels2 = np.where(y_test_pred == 1, 'positivo', 'negativo')

In [ ]:
submission = pd.DataFrame({
    'ID': test_df['ID'],
    'sentimiento': y_test_pred_labels2
})

submission.to_csv('xgboost_submission.csv', index=False)

# XGBoost (Embbedings)

Dividimos el train, en train y validation

In [137]:
X_tr, X_val, y_tr, y_val = train_test_split(
    X_train_em, Y_train_em, test_size=0.2, random_state=42, stratify=Y_train_em
)

Definimos el modelo

In [138]:
xgb_model = xgb.XGBClassifier(
    learning_rate=0.05,
    n_estimators=500,
    max_depth=6,
    gamma=2,
    subsample=0.7,
    colsample_bytree=0.7,
    reg_alpha=2,
    reg_lambda=5,
    use_label_encoder=False,
    eval_metric='logloss',
    random_state=42
)

xgb_model.fit(X_tr, y_tr)

/usr/local/lib/python3.11/dist-packages/xgboost/core.py:158: UserWarning: [11:56:19] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)


XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=0.7, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric='logloss',
              feature_types=None, gamma=2, grow_policy=None,
              importance_type=None, interaction_constraints=None,
              learning_rate=0.05, max_bin=None, max_cat_threshold=None,
              max_cat_to_onehot=None, max_delta_step=None, max_depth=6,
              max_leaves=None, min_child_weight=None, missing=nan,
              monotone_constraints=None, multi_strategy=None, n_estimators=500,
              n_jobs=None, num_parallel_tree=None, random_state=42, ...)

Evaluamos

In [139]:

y_tr_pred = xgb_model.predict(X_tr)
print("Train classification report:\n", classification_report(y_tr, y_tr_pred))
print("Train F1 Score:", f1_score(y_tr, y_tr_pred))

y_val_pred = xgb_model.predict(X_val)
print("\nValidation classification report:\n", classification_report(y_val, y_val_pred))
print("Validation F1 Score:", f1_score(y_val, y_val_pred))

Train classification report:
               precision    recall  f1-score   support

           0       0.94      0.93      0.94     20000
           1       0.93      0.94      0.94     20000

    accuracy                           0.94     40000
   macro avg       0.94      0.94      0.94     40000
weighted avg       0.94      0.94      0.94     40000

Train F1 Score: 0.9365004116869183

Validation classification report:
               precision    recall  f1-score   support

           0       0.85      0.84      0.84      5000
           1       0.84      0.85      0.84      5000

    accuracy                           0.84     10000
   macro avg       0.84      0.84      0.84     10000
weighted avg       0.84      0.84      0.84     10000

Validation F1 Score: 0.8436320286111663


Predecimos

In [140]:
y_test_pred = xgb_model.predict(X_test_em)
y_test_pred_labels = np.where(y_test_pred == 1, 'positivo', 'negativo')

In [141]:
submission = pd.DataFrame({
    'ID': test_df['ID'],
    'sentimiento': y_test_pred_labels
})

submission.to_csv('xgboost_em_submission.csv', index=False)

In [142]:
dump(xgb_model, 'xgboost.joblib')

['xgboost.joblib']

## XGBOOST (Embedings con dataset filtrado)

In [143]:
X_tr, X_val, y_tr, y_val = train_test_split(
    X_train_em2, Y_train_em2, test_size=0.2, random_state=42, stratify=Y_train_em2
)

In [144]:
xgb_model2 = xgb.XGBClassifier(
    learning_rate=0.05,
    n_estimators=500,
    max_depth=6,
    gamma=2,
    subsample=0.7,
    colsample_bytree=0.7,
    reg_alpha=2,
    reg_lambda=5,
    use_label_encoder=False,
    eval_metric='logloss',
    random_state=42
)

xgb_model2.fit(X_tr, y_tr)

/usr/local/lib/python3.11/dist-packages/xgboost/core.py:158: UserWarning: [11:56:47] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)


XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=0.7, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric='logloss',
              feature_types=None, gamma=2, grow_policy=None,
              importance_type=None, interaction_constraints=None,
              learning_rate=0.05, max_bin=None, max_cat_threshold=None,
              max_cat_to_onehot=None, max_delta_step=None, max_depth=6,
              max_leaves=None, min_child_weight=None, missing=nan,
              monotone_constraints=None, multi_strategy=None, n_estimators=500,
              n_jobs=None, num_parallel_tree=None, random_state=42, ...)

In [145]:

y_tr_pred = xgb_model2.predict(X_tr)
print("Train classification report:\n", classification_report(y_tr, y_tr_pred))
print("Train F1 Score:", f1_score(y_tr, y_tr_pred))

y_val_pred = xgb_model2.predict(X_val)
print("\nValidation classification report:\n", classification_report(y_val, y_val_pred))
print("Validation F1 Score:", f1_score(y_val, y_val_pred))

Train classification report:
               precision    recall  f1-score   support

           0       0.94      0.94      0.94     19305
           1       0.94      0.94      0.94     19239

    accuracy                           0.94     38544
   macro avg       0.94      0.94      0.94     38544
weighted avg       0.94      0.94      0.94     38544

Train F1 Score: 0.9386037814253064

Validation classification report:
               precision    recall  f1-score   support

           0       0.85      0.84      0.84      4827
           1       0.84      0.85      0.84      4810

    accuracy                           0.84      9637
   macro avg       0.84      0.84      0.84      9637
weighted avg       0.84      0.84      0.84      9637

Validation F1 Score: 0.8447563996696945


In [146]:
y_test_pred = xgb_model2.predict(X_test_em2)
y_test_pred_labels = np.where(y_test_pred == 1, 'positivo', 'negativo')

In [147]:
submission = pd.DataFrame({
    'ID': test_df['ID'],
    'sentimiento': y_test_pred_labels
})

submission.to_csv('xgboost_em2_submission.csv', index=False)

In [148]:
dump(xgb_model2, 'xbg_em2.joblib')

['xbg_em2.joblib']

# Bayes Naive

Dividimos los datos

In [86]:
X_tr, X_val, Y_tr, Y_val = train_test_split(
    X_train, Y_train, test_size=0.2, random_state=42, stratify=Y_train
)

Definimos el modelo y espacio de busqueda de hiperparametro

In [87]:
nb = MultinomialNB()
param_grid = {
    'alpha': [0.1, 0.5, 1.0, 2.0, 5.0]
}

In [88]:
grid_search = GridSearchCV(
    estimator=nb,
    param_grid=param_grid,
    scoring='f1',
    cv=3,
    verbose=1,
    n_jobs=-1
)

Realizar busqueda

In [89]:
grid_search.fit(X_tr, Y_tr)
best_nb = grid_search.best_estimator_

print("Mejor alpha: ", grid_search.best_params_['alpha'])
print("Mejor F1: ", grid_search.best_score_)

Fitting 3 folds for each of 5 candidates, totalling 15 fits
Mejor alpha:  0.1
Mejor F1:  0.8414240275262227


Evaluamos modelo en train y validation

In [90]:
y_tr_pred = best_nb.predict(X_tr)
y_val_pred = best_nb.predict(X_val)

print("\n Train classification report:\n", classification_report(Y_tr, y_tr_pred))
print("Train F1 Score:", f1_score(Y_tr, y_tr_pred))

print("\n Validation classification report:\n", classification_report(Y_val, y_val_pred))
print("Validation F1 Score:", f1_score(Y_val, y_val_pred))


 Train classification report:
               precision    recall  f1-score   support

           0       0.87      0.83      0.85     20000
           1       0.84      0.87      0.85     20000

    accuracy                           0.85     40000
   macro avg       0.85      0.85      0.85     40000
weighted avg       0.85      0.85      0.85     40000

Train F1 Score: 0.8542479330732808

 Validation classification report:
               precision    recall  f1-score   support

           0       0.87      0.82      0.84      5000
           1       0.83      0.87      0.85      5000

    accuracy                           0.85     10000
   macro avg       0.85      0.85      0.85     10000
weighted avg       0.85      0.85      0.85     10000

Validation F1 Score: 0.8503998439633313


Predecir en test

In [91]:
y_test_pred = best_nb.predict(X_test)
y_test_pred_labels = np.where(y_test_pred == 1, 'positivo', 'negativo')

Generar prediccion

In [92]:
submission_nb = pd.DataFrame({
    'ID': test_df['ID'],
    'sentimiento': y_test_pred_labels
})
submission_nb.to_csv('nb_submission.csv', index=False)

In [93]:
dump(best_nb, 'bayes_naive.joblib')

['bayes_naive.joblib']

# Bayes Naive v2

Kaggle: 0.74122

In [ ]:
X_tr, X_val, Y_tr, Y_val = train_test_split(
    X_train, Y_train, test_size=0.2, random_state=42, stratify=Y_train
)

In [ ]:
model = MultinomialNB()
model.fit(X_tr, Y_tr)

MultinomialNB()

In [ ]:
y_pred = model.predict(X_val)

from sklearn.metrics import f1_score, classification_report
f1 = f1_score(Y_val, y_pred)
print(f"F1-score: {f1:.4f}")
print(classification_report(Y_val, y_pred))

F1-score: 0.8490
              precision    recall  f1-score   support

           0       0.86      0.82      0.84      5000
           1       0.83      0.87      0.85      5000

    accuracy                           0.85     10000
   macro avg       0.85      0.85      0.85     10000
weighted avg       0.85      0.85      0.85     10000



In [ ]:
model_train = MultinomialNB()
model_train.fit(X_train, Y_train)

y_train_pred = model_train.predict(X_train)

print("Entrenamiento:")
print(classification_report(Y_train, y_train_pred))
print(f"F1-score entrenamiento: {f1_score(Y_train, y_train_pred):.4f}")


Entrenamiento:
              precision    recall  f1-score   support

           0       0.86      0.83      0.85     25000
           1       0.83      0.87      0.85     25000

    accuracy                           0.85     50000
   macro avg       0.85      0.85      0.85     50000
weighted avg       0.85      0.85      0.85     50000

F1-score entrenamiento: 0.8512


In [ ]:
y_test_pred = model.predict(X_test)
y_test_pred_labels = np.where(y_test_pred == 1, 'positivo', 'negativo')

In [ ]:
dump(model, 'bn_v2.joblib')

['bn_v2.joblib']

In [ ]:
submission_nb = pd.DataFrame({
    'ID': test_df['ID'],
    'sentimiento': y_test_pred_labels
})

submission_nb.to_csv('nb_submission.csv', index=False)

# Bayes Naive (Lematización)

In [ ]:
import nltk
from nltk.corpus import stopwords

nltk.download('stopwords')
stop_words = set(stopwords.words('spanish'))

print(stop_words)

{'habremos', 'estuviese', 'seríais', 'esto', 'ti', 'seáis', 'sus', 'son', 'eras', 'él', 'estado', 'hay', 'cual', 'todo', 'hubiste', 'habré', 'los', 'fuésemos', 'ni', 'tuyos', 'míos', 'estuvieras', 'habíamos', 'serían', 'a', 'donde', 'o', 'fuese', 'hayas', 'estados', 'tuviera', 'hemos', 'estuviéramos', 'tenías', 'nuestros', 'esa', 'está', 'habías', 'habiendo', 'más', 'habrán', 'algo', 'tuvimos', 'e', 'tened', 'tendré', 'pero', 'han', 'habrían', 'serías', 'estamos', 'sentido', 'tengan', 'tuviésemos', 'fueseis', 'estás', 'entre', 'habíais', 'habrá', 'tenidos', 'tendremos', 'estáis', 'hubiesen', 'tus', 'habrías', 'el', 'ellas', 'ante', 'muchos', 'estuviera', 'es', 'hubieras', 'estabais', 'hubiéramos', 'ese', 'era', 'fueran', 'habidos', 'tengáis', 'tuviéramos', 'hube', 'estaban', 'quienes', 'están', 'esos', 'cuando', 'que', 'había', 'estada', 'tuvieran', 'nuestra', 'sentid', 'con', 'un', 'tenido', 'mucho', 'sentida', 'estéis', 'su', 'estarían', 'quien', 'nosotras', 'este', 'hubiera', 'fuera

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [ ]:
stopwords_a_conservar = {'no', 'ni', 'nada', 'nunca', 'jamás', 'sin', 'muy', 'poco', 'se', 'más', 'menos', 'pero', 'aunque', 'contra', 'sentido', 'algunos', 'todo', 'tanto'}

stop_words = stop_words - stopwords_a_conservar

In [ ]:
nlp = spacy.load('es_core_news_sm')

def lematizar_filtrar(texto):
    doc = nlp(texto)
    lemas = [
        token.lemma_ for token in doc
        if token.lemma_ not in stop_words and not token.is_punct and not token.is_space
    ]
    return ' '.join(lemas)

train_df['review_clean'] = train_df['review_clean'].apply(lematizar_filtrar)
test_df['review_clean'] = test_df['review_clean'].apply(lematizar_filtrar)

In [ ]:
X_tr, X_val, y_tr, y_val = train_test_split(
    train_df['review_clean'],
    train_df['sentimiento_binario'],
    test_size=0.2,
    stratify=train_df['sentimiento_binario'],
    random_state=42
)

In [ ]:
from sklearn.pipeline import Pipeline

pipeline = Pipeline([
    ('tfidf', TfidfVectorizer()),
    ('clf', MultinomialNB())
])

In [ ]:
param_grid = {
    'tfidf__max_features': [5000, 10000, 15000],
    'tfidf__min_df': [3, 5, 10],
    'tfidf__ngram_range': [(1,1), (1,2), (1,3)],
    'clf__alpha': [0.1, 0.3]
}

In [ ]:
grid_search = GridSearchCV(
    pipeline,
    param_grid,
    cv=3,
    scoring='f1',
    verbose=1,
    n_jobs=-1
)

grid_search.fit(X_tr, y_tr)

Fitting 3 folds for each of 54 candidates, totalling 162 fits


GridSearchCV(cv=3,
             estimator=Pipeline(steps=[('tfidf', TfidfVectorizer()),
                                       ('clf', MultinomialNB())]),
             n_jobs=-1,
             param_grid={'clf__alpha': [0.1, 0.3],
                         'tfidf__max_features': [5000, 10000, 15000],
                         'tfidf__min_df': [3, 5, 10],
                         'tfidf__ngram_range': [(1, 1), (1, 2), (1, 3)]},
             scoring='f1', verbose=1)

In [ ]:
print("Mejores parámetros:", grid_search.best_params_)
print("Mejor F1:", grid_search.best_score_)

Mejores parámetros: {'clf__alpha': 0.1, 'tfidf__max_features': 15000, 'tfidf__min_df': 5, 'tfidf__ngram_range': (1, 3)}
Mejor F1: 0.8540445368110078


In [ ]:
best_model = grid_search.best_estimator_
y_tr_pred = best_model.predict(X_tr)
y_val_pred = best_model.predict(X_val)


print("\n Train classification report:\n", classification_report(y_tr, y_tr_pred))
print("Train F1 Score:", f1_score(y_tr, y_tr_pred))

print("\n Validation classification report:\n", classification_report(y_val, y_val_pred))
print("Validation F1 Score:", f1_score(y_val, y_val_pred))


 Train classification report:
               precision    recall  f1-score   support

           0       0.89      0.85      0.87     20000
           1       0.86      0.89      0.87     20000

    accuracy                           0.87     40000
   macro avg       0.87      0.87      0.87     40000
weighted avg       0.87      0.87      0.87     40000

Train F1 Score: 0.87317730670261

 Validation classification report:
               precision    recall  f1-score   support

           0       0.88      0.84      0.86      5000
           1       0.84      0.89      0.86      5000

    accuracy                           0.86     10000
   macro avg       0.86      0.86      0.86     10000
weighted avg       0.86      0.86      0.86     10000

Validation F1 Score: 0.8643422209200117


In [ ]:
y_test_pred = best_model.predict(test_df['review_clean'])
y_test_pred_labels = np.where(y_test_pred == 1, 'positivo', 'negativo')

In [ ]:
submission_nb = pd.DataFrame({
    'ID': test_df['ID'],
    'sentimiento': y_test_pred_labels
})
submission_nb.to_csv('nb_l_submission.csv', index=False)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

# Bayes Naive v3

(Modelo no subido a la competencia)

In [ ]:
X_tr, X_val, Y_tr, Y_val = train_test_split(
    X_train, Y_train, test_size=0.2, random_state=42, stratify=Y_train
)

In [ ]:
TfidfVectorizer(
    max_features=10000,
    ngram_range=(1,2),
    min_df=5,
    max_df=0.9,
)

TfidfVectorizer(max_df=0.9, max_features=10000, min_df=5, ngram_range=(1, 2))

In [ ]:
params = {'alpha': [0.01, 0.1, 0.5, 1.0, 2.0, 5.0]}
nb = MultinomialNB()
grid = GridSearchCV(nb, param_grid=params, cv=5, scoring='f1', n_jobs=-1)
grid.fit(X_train, Y_train)

print(f"Mejor alpha: {grid.best_params_}")
print(f"Mejor F1: {grid.best_score_:.4f}")

Mejor alpha: {'alpha': 0.01}
Mejor F1: 0.8431


Peor desemepelo que el v2. No se sube

# Random Forest

Dividimos los datos

In [ ]:
X_tr, X_val, y_tr, y_val = train_test_split(
    X_train, Y_train, test_size=0.2, random_state=42, stratify=Y_train
)

Definimos modelo y busqueda de hiperparametros

In [ ]:
rf = RandomForestClassifier(random_state=42)

In [ ]:
param_distributions = {
    'n_estimators': [200, 300, 500],
    'max_depth': [5, 10, 15],
    'max_features': ['sqrt', 'log2'],
    'min_samples_split': [5, 10, 20],
    'min_samples_leaf': [2, 4, 6],
    'bootstrap': [True, False]
}

In [ ]:
random_search_rf = RandomizedSearchCV(
    estimator=rf,
    param_distributions=param_distributions,
    n_iter=10,
    scoring='f1',
    cv=3,
    verbose=2,
    random_state=42,
    n_jobs=-1
)

Busqueda

In [ ]:
random_search_rf.fit(X_tr, y_tr)
best_rf = random_search_rf.best_estimator_

Fitting 3 folds for each of 10 candidates, totalling 30 fits


In [ ]:
print("Mejores hiperparámetros:", random_search_rf.best_params_)
print("Mejor F1:", random_search_rf.best_score_)

Mejores hiperparámetros: {'n_estimators': 500, 'min_samples_split': 5, 'min_samples_leaf': 2, 'max_features': 'log2', 'max_depth': 15, 'bootstrap': True}
Mejor F1: 0.8254012729286583


Evaluamos

In [ ]:
y_tr_pred = best_rf.predict(X_tr)
y_val_pred = best_rf.predict(X_val)

In [ ]:
print("\n Train classification report:\n", classification_report(Y_tr, y_tr_pred))
print("Train F1 Score:", f1_score(Y_tr, y_tr_pred))

print("\n Validation classification report:\n", classification_report(Y_val, y_val_pred))
print("Validation F1 Score:", f1_score(Y_val, y_val_pred))


 Train classification report:
               precision    recall  f1-score   support

           0       0.92      0.82      0.87     20000
           1       0.84      0.93      0.88     20000

    accuracy                           0.87     40000
   macro avg       0.88      0.87      0.87     40000
weighted avg       0.88      0.87      0.87     40000

Train F1 Score: 0.8795883524613488

 Validation classification report:
               precision    recall  f1-score   support

           0       0.86      0.77      0.81      5000
           1       0.79      0.87      0.83      5000

    accuracy                           0.82     10000
   macro avg       0.82      0.82      0.82     10000
weighted avg       0.82      0.82      0.82     10000

Validation F1 Score: 0.8301779088573875


Prediccion

In [ ]:
y_test_pred = best_rf.predict(X_test)
y_test_pred_labels = np.where(y_test_pred == 1, 'positivo', 'negativo')

In [ ]:
submission_rf = pd.DataFrame({
    'ID': test_df['ID'],
    'label': y_test_pred_labels
})
submission_rf.to_csv('random_forest_submission.csv', index=False)

In [ ]:
dump(best_rf, 'random_forest.joblib')

['random_forest.joblib']

# Random Forest v2

Kaggle: 0.68191

Random Forest con distintos hiperparámetros

In [ ]:
X_tr, X_val, Y_tr, Y_val = train_test_split(
    X_train, Y_train, test_size=0.2, random_state=42, stratify=Y_train
)

In [ ]:
rfc =  RandomForestClassifier(max_features='sqrt',
                             oob_score=True,
                             random_state=2,
                             n_jobs=-1,
                             criterion="entropy",
                             min_samples_leaf=5,
                             min_samples_split=5,
                             n_estimators=50 )

In [ ]:
model_rfc = rfc.fit(X = X_tr, y = Y_tr)

In [ ]:
y_pred = model_rfc.predict(X_val)
y_pred

array([1, 0, 1, ..., 0, 1, 1])

In [ ]:
Y_val.values

array([0, 0, 1, ..., 0, 1, 0])

In [ ]:
f1=f1_score(Y_val,y_pred)
print("f1 score: "+str(f1))

f1 score: 0.8244244639857722


In [ ]:
dump(model_rfc, 'random_forest_v2.joblib')

['random_forest_v2.joblib']

In [ ]:
y_test_pred = model_rfc.predict(X_test)
y_test_pred_labels = np.where(y_test_pred == 1, 'positivo', 'negativo')

submission_rf = pd.DataFrame({
    'ID': test_df['ID'],
    'sentimiento': y_test_pred_labels
})

submission_rf.to_csv('rf_submission.csv', index=False)

# Red Neuronal

Divison y armado de arrays

In [ ]:
X_tr, X_val, y_tr, y_val = train_test_split(X_train, Y_train, test_size=0.2, stratify=Y_train, random_state=42)
X_tr_dense = X_tr.toarray()
X_val_dense = X_val.toarray()
X_test_dense = X_test.toarray()

Construccion del modelo

In [ ]:
def build_model(hp):
    model = Sequential()
    model.add(Dense(
        units=hp.Int('units_input', 64, 192, step=32),
        activation=hp.Choice('act_input', ['relu', 'swish']),
        input_shape=(X_tr_dense.shape[1],)
    ))
    model.add(BatchNormalization())
    model.add(Dropout(hp.Float('drop_input', 0.2, 0.4, step=0.1)))

    for i in range(hp.Int('num_layers', 1, 2)):  # solo 1 o 2 capas ocultas
        model.add(Dense(
            units=hp.Int(f'units_{i}', 32, 128, step=32),
            activation=hp.Choice(f'act_{i}', ['relu', 'swish'])
        ))
        model.add(BatchNormalization())
        model.add(Dropout(hp.Float(f'dropout_{i}', 0.2, 0.5, step=0.1)))

    model.add(Dense(1, activation='sigmoid'))
    model.compile(
        optimizer=Adam(learning_rate=hp.Choice('lr', [1e-2, 5e-3, 1e-3])),
        loss='binary_crossentropy',
        metrics=['accuracy']
    )
    return model

Tuner

In [ ]:
tuner = kt.RandomSearch(
    build_model,
    objective='val_accuracy',
    max_trials=10,
    executions_per_trial=1,
    directory='nn_tuning_final',
    project_name='nn_max_fit'
)

/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Callbacks

In [ ]:
early_stop = EarlyStopping(patience=3, restore_best_weights=True)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=2, verbose=1)

Tuning

In [ ]:
tuner.search(
    X_tr_dense, y_tr,
    validation_data=(X_val_dense, y_val),
    epochs=15,
    batch_size=64,
    callbacks=[early_stop, reduce_lr],
    verbose=1
)

Trial 10 Complete [00h 00m 23s]
val_accuracy: 0.8762000203132629

Best val_accuracy So Far: 0.8762999773025513
Total elapsed time: 00h 05m 26s


Mejor modelo

In [ ]:
best_nn = tuner.get_best_models(1)[0]
y_val_pred = (best_nn.predict(X_val_dense) > 0.5).astype(int).flatten()
print("Validation F1 Score:", f1_score(y_val, y_val_pred))

/usr/local/lib/python3.11/dist-packages/keras/src/saving/saving_lib.py:757: UserWarning: Skipping variable loading for optimizer 'adam', because it has 2 variables whereas the saved optimizer has 22 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


313/313 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step
Validation F1 Score: 0.8779717865246128


Prediccion final

In [ ]:
y_test_pred = (best_nn.predict(X_test_dense) > 0.5).astype(int).flatten()
y_test_labels = np.where(y_test_pred == 1, 'positivo', 'negativo')

submission_nn = pd.DataFrame({
    'ID': test_df['ID'],
    'sentimiento': y_test_labels
})
submission_nn.to_csv('rn3_submission.csv', index=False)

In [ ]:
dump(best_nn, 'red_neuronal.joblib')

['red_neuronal.joblib']

In [ ]:
y_test_pred = (best_nn.predict(X_test_dense) > 0.5).astype(int).flatten()
y_test_labels = np.where(y_test_pred == 1, 'positivo', 'negativo')

submission_nn = pd.DataFrame({
    'ID': test_df['ID'],
    'sentimiento': y_test_labels
})
submission_nn.to_csv('rn3_submission.csv', index=False)

# Red Neuronal v2

Kaggle: 0.73812

In [ ]:
X_tr, X_val, y_tr, y_val = train_test_split(X_train, Y_train, test_size=0.2, stratify=Y_train, random_state=42)
X_tr_dense = X_tr.toarray()
X_val_dense = X_val.toarray()
X_test_dense = X_test.toarray()

In [ ]:
model = Sequential()
model.add(Dense(128, input_shape=(X_train.shape[1],), activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(64, activation='relu'))
model.add(Dense(1, activation='sigmoid'))

model.compile(optimizer=Adam(learning_rate=0.001),
              loss='binary_crossentropy',
              metrics=['accuracy'])

/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [ ]:
early_stop = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)

model.fit(
    X_train, Y_train,
    validation_split=0.2,
    epochs=20,
    batch_size=32,
    callbacks=[early_stop],
    verbose=2
)

Epoch 1/20
1250/1250 - 855s - 684ms/step - accuracy: 0.8426 - loss: 0.3513 - val_accuracy: 0.8727 - val_loss: 0.2937
Epoch 2/20
1250/1250 - 458s - 366ms/step - accuracy: 0.9083 - loss: 0.2273 - val_accuracy: 0.8718 - val_loss: 0.2972
Epoch 3/20
1250/1250 - 277s - 222ms/step - accuracy: 0.9375 - loss: 0.1656 - val_accuracy: 0.8721 - val_loss: 0.3274
Epoch 4/20
1250/1250 - 210s - 168ms/step - accuracy: 0.9633 - loss: 0.1039 - val_accuracy: 0.8656 - val_loss: 0.4031


In [ ]:
y_val_pred_prob = model.predict(X_val)
y_val_pred = (y_val_pred_prob >= 0.5).astype(int)

313/313 ━━━━━━━━━━━━━━━━━━━━ 32s 103ms/step


In [ ]:
print("F1 Score:", f1_score(y_val, y_val_pred))
print("Reporte completo:\n", classification_report(y_val, y_val_pred))

F1 Score: 0.9167259786476868
Reporte completo:
               precision    recall  f1-score   support

           0       0.90      0.93      0.92      5000
           1       0.93      0.90      0.92      5000

    accuracy                           0.92     10000
   macro avg       0.92      0.92      0.92     10000
weighted avg       0.92      0.92      0.92     10000



In [ ]:
y_tr_pred = (model.predict(X_tr).flatten() >= 0.5).astype(int)
print("Train F1 Score:", f1_score(y_tr, y_tr_pred))

1250/1250 ━━━━━━━━━━━━━━━━━━━━ 91s 73ms/step
Train F1 Score: 0.9121094246561412


In [ ]:
dump(model, 'red_neuronal_v2.joblib')

['red_neuronal_v2.joblib']

In [ ]:
y_test_pred_prob = model.predict(X_test)
y_test_pred = (y_test_pred_prob >= 0.5).astype(int)
y_test_labels = np.where(y_test_pred == 1, 'positivo', 'negativo')

submission_nn = pd.DataFrame({
    'ID': test_df['ID'],
    'sentimiento': y_test_labels.flatten()
})
submission_nn.to_csv("rn_v2_submission.csv", index=False)

269/269 ━━━━━━━━━━━━━━━━━━━━ 41s 151ms/step


# Red Neuronal (Embbedings)

Divison y armado de arrays

In [49]:
X_tr, X_val, y_tr, y_val = train_test_split(X_train_em, Y_train_em, test_size=0.2, stratify=Y_train_em, random_state=42)
X_tr_dense = X_tr
X_val_dense = X_val
X_test_dense = X_test_em

Construccion del modelo

In [50]:
def build_model(hp):
    model = Sequential()
    model.add(Dense(
        units=hp.Int('units_input', 64, 192, step=32),
        activation=hp.Choice('act_input', ['relu', 'swish']),
        input_shape=(X_tr_dense.shape[1],)
    ))
    model.add(BatchNormalization())
    model.add(Dropout(hp.Float('drop_input', 0.2, 0.4, step=0.1)))

    for i in range(hp.Int('num_layers', 1, 2)):  # solo 1 o 2 capas ocultas
        model.add(Dense(
            units=hp.Int(f'units_{i}', 32, 128, step=32),
            activation=hp.Choice(f'act_{i}', ['relu', 'swish'])
        ))
        model.add(BatchNormalization())
        model.add(Dropout(hp.Float(f'dropout_{i}', 0.2, 0.5, step=0.1)))

    model.add(Dense(1, activation='sigmoid'))
    model.compile(
        optimizer=Adam(learning_rate=hp.Choice('lr', [1e-2, 5e-3, 1e-3])),
        loss='binary_crossentropy',
        metrics=['accuracy']
    )
    return model

Tuner

In [51]:
tuner = kt.RandomSearch(
    build_model,
    objective='val_accuracy',
    max_trials=10,
    executions_per_trial=1,
    directory='nn_tuning_final',
    project_name='nn_max_fit'
)

/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Callbacks

In [52]:
early_stop = EarlyStopping(patience=3, restore_best_weights=True)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=2, verbose=1)

Tuning

In [53]:
tuner.search(
    X_tr_dense, y_tr,
    validation_data=(X_val_dense, y_val),
    epochs=15,
    batch_size=64,
    callbacks=[early_stop, reduce_lr],
    verbose=1
)

Trial 10 Complete [00h 01m 00s]
val_accuracy: 0.8583999872207642

Best val_accuracy So Far: 0.8610000014305115
Total elapsed time: 00h 09m 07s


Mejor modelo

In [54]:
best_nn = tuner.get_best_models(1)[0]
y_val_pred = (best_nn.predict(X_val_dense) > 0.5).astype(int).flatten()
print("Validation F1 Score:", f1_score(y_val, y_val_pred))

  1/313 ━━━━━━━━━━━━━━━━━━━━ 31s 100ms/step

/usr/local/lib/python3.11/dist-packages/keras/src/saving/saving_lib.py:757: UserWarning: Skipping variable loading for optimizer 'adam', because it has 2 variables whereas the saved optimizer has 22 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


313/313 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step
Validation F1 Score: 0.8614157527417746


In [55]:
print("\nClassification Report:\n", classification_report(y_val, y_val_pred))


Classification Report:
               precision    recall  f1-score   support

           0       0.86      0.86      0.86      5000
           1       0.86      0.86      0.86      5000

    accuracy                           0.86     10000
   macro avg       0.86      0.86      0.86     10000
weighted avg       0.86      0.86      0.86     10000



Prediccion final

In [56]:
y_test_pred = (best_nn.predict(X_test_dense) > 0.5).astype(int).flatten()
y_test_labels = np.where(y_test_pred == 1, 'positivo', 'negativo')

submission_nn = pd.DataFrame({
    'ID': test_df['ID'],
    'sentimiento': y_test_labels
})
submission_nn.to_csv('rn_em_submission.csv', index=False)

269/269 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step


In [57]:
dump(best_nn, 'red_neuronal_em.joblib')

['red_neuronal_em.joblib']

## Red Neuronal (Embeddings con dataset filtrado)

In [76]:
X_tr, X_val, y_tr, y_val = train_test_split(X_train_em2, Y_train_em2, test_size=0.2, stratify=Y_train_em2, random_state=42)
X_tr_dense2 = X_tr
X_val_dense2 = X_val
X_test_dense2 = X_test_em

In [77]:
def build_model(hp):
    model = Sequential()
    model.add(Dense(
        units=hp.Int('units_input', 64, 192, step=32),
        activation=hp.Choice('act_input', ['relu', 'swish']),
        input_shape=(X_tr_dense2.shape[1],)
    ))
    model.add(BatchNormalization())
    model.add(Dropout(hp.Float('drop_input', 0.2, 0.4, step=0.1)))

    for i in range(hp.Int('num_layers', 1, 2)):  # solo 1 o 2 capas ocultas
        model.add(Dense(
            units=hp.Int(f'units_{i}', 32, 128, step=32),
            activation=hp.Choice(f'act_{i}', ['relu', 'swish'])
        ))
        model.add(BatchNormalization())
        model.add(Dropout(hp.Float(f'dropout_{i}', 0.2, 0.5, step=0.1)))

    model.add(Dense(1, activation='sigmoid'))
    model.compile(
        optimizer=Adam(learning_rate=hp.Choice('lr', [1e-2, 5e-3, 1e-3])),
        loss='binary_crossentropy',
        metrics=['accuracy']
    )
    return model

In [78]:
tuner = kt.RandomSearch(
    build_model,
    objective='val_accuracy',
    max_trials=10,
    executions_per_trial=1,
    directory='nn_tuning_final',
    project_name='nn_max_fit2'
)

/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [79]:
early_stop = EarlyStopping(patience=3, restore_best_weights=True)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=2, verbose=1)

In [80]:
tuner.search(
    X_tr_dense2, y_tr,
    validation_data=(X_val_dense2, y_val),
    epochs=15,
    batch_size=64,
    callbacks=[early_stop, reduce_lr],
    verbose=1
)

Trial 10 Complete [00h 00m 49s]
val_accuracy: 0.8551416397094727

Best val_accuracy So Far: 0.8580471277236938
Total elapsed time: 00h 08m 38s


In [81]:
best_nn = tuner.get_best_models(1)[0]
y_val_pred = (best_nn.predict(X_val_dense2) > 0.5).astype(int).flatten()
print("Validation F1 Score:", f1_score(y_val, y_val_pred))

  1/302 ━━━━━━━━━━━━━━━━━━━━ 30s 102ms/step

/usr/local/lib/python3.11/dist-packages/keras/src/saving/saving_lib.py:757: UserWarning: Skipping variable loading for optimizer 'adam', because it has 2 variables whereas the saved optimizer has 22 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


302/302 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step
Validation F1 Score: 0.8559696778269109


In [82]:
print("\nClassification Report:\n", classification_report(y_val, y_val_pred))


Classification Report:
               precision    recall  f1-score   support

           0       0.85      0.87      0.86      4827
           1       0.87      0.85      0.86      4810

    accuracy                           0.86      9637
   macro avg       0.86      0.86      0.86      9637
weighted avg       0.86      0.86      0.86      9637



In [104]:
y_test_pred = (best_nn.predict(X_test_dense2) > 0.5).astype(int).flatten()
y_test_labels = np.where(y_test_pred == 1, 'positivo', 'negativo')

submission_nn = pd.DataFrame({
    'ID': test_df['ID'],
    'sentimiento': y_test_labels
})
submission_nn.to_csv('rn_em2_submission.csv', index=False)

269/269 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step


In [103]:
dump(best_nn, 'red_neuronal_em2.joblib')

['red_neuronal_em2.joblib']

# Ensamble

Convertimos los sentimientos a binario

In [ ]:
def str_to_bin(arr):
    return np.where(np.array(arr) == 'positivo', 1, 0)

random_forest_submission = pd.read_csv('random_forest_submission.csv')['sentimiento']
nb_submission = pd.read_csv('nb_submission.csv')['sentimiento']
rn_submission = pd.read_csv('rn_submission.csv')['sentimiento']

bin_xgb = str_to_bin(random_forest_submission)
bin_rf = str_to_bin(nb_submission)
bin_nn = str_to_bin(rn_submission)

Apilar y votar

In [ ]:
preds = np.vstack([bin_xgb, bin_rf, bin_nn]).T
y_test_ensemble = np.round(preds.mean(axis=1)).astype(int)

Volver a convertir a sentimiento

In [ ]:
y_test_labels_ensemble = np.where(y_test_ensemble == 1, 'positivo', 'negativo')

Ensamble final

In [ ]:
submission_ensemble = pd.DataFrame({
    'ID': test_df['ID'],
    'sentimiento': y_test_labels_ensemble
})
submission_ensemble.to_csv('ensemble2_submission.csv', index=False)

# Ensamble (RN (Embbedings), XGBoost(Embbedings), BN)

Convertimos los sentimientos a binario

In [114]:
def str_to_bin(arr):
    return np.where(np.array(arr) == 'positivo', 1, 0)

xgboost_submission = pd.read_csv('nb_submission.csv')['sentimiento']
nb_submission = pd.read_csv('xgboost_em_submission.csv')['sentimiento']
rn_submission = pd.read_csv('rn_em_submission.csv')['sentimiento']

bin_xgb = str_to_bin(xgboost_submission)
bin_rf = str_to_bin(nb_submission)
bin_nn = str_to_bin(rn_submission)

Apilar y votar

In [115]:
preds = np.vstack([bin_xgb, bin_rf, bin_nn]).T
y_test_ensemble = np.round(preds.mean(axis=1)).astype(int)

Volver a convertir a sentimiento

In [116]:
y_test_labels_ensemble = np.where(y_test_ensemble == 1, 'positivo', 'negativo')

Ensamble final

In [117]:
submission_ensemble = pd.DataFrame({
    'ID': test_df['ID'],
    'sentimiento': y_test_labels_ensemble
})
submission_ensemble.to_csv('ensemble_submission.csv', index=False)

## Ensamble (RN (Embbedings data filtrada), XGBoost(Embbedings), BN)

In [118]:
X_tr, X_val, y_tr, y_val = train_test_split(X_train_em2, Y_train_em2, test_size=0.2, stratify=Y_train_em2, random_state=42)
X_tr_dense2 = X_tr
X_val_dense2 = X_val
X_test_dense2 = X_test_em

In [119]:
def str_to_bin(arr):
    return np.where(np.array(arr) == 'positivo', 1, 0)

random_forest_submission = pd.read_csv('nb_submission.csv')['sentimiento']
nb_submission = pd.read_csv('xgboost_em_submission.csv')['sentimiento']
rn_submission = pd.read_csv('rn_em2_submission.csv')['sentimiento']

bin_xgb = str_to_bin(random_forest_submission)
bin_rf = str_to_bin(nb_submission)
bin_nn = str_to_bin(rn_submission)

In [120]:
preds = np.vstack([bin_xgb, bin_rf, bin_nn]).T
y_test_ensemble = np.round(preds.mean(axis=1)).astype(int)

In [121]:
y_test_labels_ensemble = np.where(y_test_ensemble == 1, 'positivo', 'negativo')

In [122]:
submission_ensemble = pd.DataFrame({
    'ID': test_df['ID'],
    'sentimiento': y_test_labels_ensemble
})
submission_ensemble.to_csv('ensemble3_submission.csv', index=False)

Kaggle: 0.75537